### Uso de las funciones.
En el archivo codigo_EnriqueSantibanez.py están las funciones (comentadas) para resolver un problema combinatorio utilizando un método constructivo aleatorio.
- load_parametres(paht): Esta función carga los parametros del problema.
- semi_greedy(*): Resuelve el problema

In [1]:
# load library
from codigo_EnriqueSantibanez import *

In [2]:
def main():
    # load parameters
    path = "../data/datos_EnriqueSantibanez.txt"
    p, alpha, c_ij, f_i, instacias = load_parametres(path)
    
    # method constructive
    y_opt, A_opti, cost_final = semi_greedy(instacias, c_ij, f_i, p, alpha=0.1)

In [3]:
if __name__ == "__main__":
    main()

-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
-------------------------Solución construida---------------------------------
Las instalaciones que se tiene que abrir son [ 7  9 11 10]
Las asignaciones de las instalaciones clientes son:
[[1. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
-----------------Evaluación de la función objetivo---------------------------
El costo final es: 645


## Comparación de las soluciones utilizando la libraría pulp

Para comparar nuestros resultados obtenidos con las funciones programadas, utilizamos la librería pulp para encontrar la solución óptima del problema basandonos de los ejemplos de la documentación (https://coin-or.github.io/pulp/CaseStudies/a_set_partitioning_problem.html).

In [4]:
# load library
from pulp import *

In [5]:
# Lists (sets / Array) of Customers and Facilities
Customer = [1,2,3,4,5,6]
Facility = ["Fac-{}".format(i) for i in range(1, 12)]

# Dictionaries of the demands and facilities as well as fixed cost at each facility
fixed_cost = {'Fac-1' : 214, 'Fac-2' : 225, 'Fac-3' : 285,
             "Fac-4":205, "Fac-5":229, "Fac-6":294, "Fac-7":137,
             "Fac-8":292, "Fac-9":155, "Fac-10":166, "Fac-11":154}

# Dictionary of the transportation cost from each facility to all the customers 
transportation_cost = {'Fac-1' : {1:18, 2:1, 3:16, 4:6, 5:10, 6:15},
                       'Fac-2' : {1:3, 2:10, 3:17, 4:13, 5:6, 6:20},
                       'Fac-3' : {1:13, 2:6, 3:12, 4:5, 5:7, 6:10},
                       'Fac-4' : {1:9, 2:14, 3:20, 4:11, 5:16, 6:2},
                       'Fac-5' : {1:13, 2:15, 3:16, 4:12, 5:11, 6:19},
                       'Fac-6' : {1:9, 2:3, 3:16, 4:16, 5:20, 6:12},
                       'Fac-7' : {1:3, 2:7, 3:9, 4:6, 5:12, 6:15},
                       'Fac-8' : {1:2, 2:12, 3:3, 4:7, 5:3, 6:16},
                       'Fac-9' : {1:4, 2:2, 3:11, 4:12, 5:5, 6:16},
                       'Fac-10' : {1:12, 2:7, 3:10, 4:18, 5:12, 6:8},
                       'Fac-11' : {1:9, 2:8, 3:19, 4:18, 5:15, 6:20}
                      }


# Setting the Problem
prob = LpProblem("Capacitated Facility Location Problem", LpMinimize)


# Defining our Desicion Variables
use_facility = LpVariable.dicts("Use Facility", Facility, 0, 1, LpBinary)
ser_customer = LpVariable.dicts("Service", [(i,j) for i in Customer for j in Facility], 0)


# Setting the Objective Function
prob += lpSum(fixed_cost[j]*use_facility[j] for j in Facility) + lpSum(transportation_cost[j][i]*ser_customer[(i,j)] for j in Facility for i in Customer)

# Costraints
for i in Customer:
    prob += lpSum(ser_customer[(i,j)] for j in Facility) == 1

for i in Customer:
    for j in Facility:
        prob += ser_customer[(i,j)] <= use_facility[j]
        
prob += lpSum(use_facility[j] for j in Facility) == 4

prob.solve()


/home/enrique/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [6]:
# Print the solution of Binary Decision Variables
Tolerance = 0.0001
print("Instancias optimas")
for j in Facility:
    if use_facility[j].varValue > Tolerance:
        print(j[4:])

Instancias optimas
7
9
10
11


In [7]:
# Print the solution of Continuous Decision Variables
for v in prob.variables():
    if v.varValue==1.:
        print(v.name, "=", v.varValue)

Service_(1,_'Fac_7') = 1.0
Service_(2,_'Fac_9') = 1.0
Service_(3,_'Fac_7') = 1.0
Service_(4,_'Fac_7') = 1.0
Service_(5,_'Fac_9') = 1.0
Service_(6,_'Fac_10') = 1.0
Use_Facility_Fac_10 = 1.0
Use_Facility_Fac_11 = 1.0
Use_Facility_Fac_7 = 1.0
Use_Facility_Fac_9 = 1.0


In [8]:
# Print Optimal
print("Total Cost = ", value(prob.objective))

Total Cost =  645.0


### Podemos observar que la solución optima es la solución que encontramos utilizando alpha muy pequeño en nuestras funciones. Lo cual tiene sentido, debido a que entre más pequeño sea alpha las soluciones obtenidas no son muy aleatorias por que lo podemos caer en un mínimo local.